# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [4]:
# Your code here

import pandas as pd
import sqlite3

#Connecting database
conn = sqlite3.connect("data.sqlite")
cur = conn.cursor()

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [6]:
# Your code here

#Selecting names of all employees in Boston
name_of_employees = '''
SELECT e.firstName, e.lastName
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
WHERE o.city = 'Boston';
'''
df = pd.read_sql_query(name_of_employees, conn)
print(df)

  firstName   lastName
0     Julie   Firrelli
1     Steve  Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [8]:
# Your code here

offices_with_zero_employees = '''
SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS num_employees
FROM offices o
LEFT JOIN employees e ON o.officeCode = e.officeCode
GROUP BY o.officeCode
HAVING num_employees = 0;
'''
df = pd.read_sql_query(offices_with_zero_employees, conn)
print(df)

   officeCode    city  num_employees
0          27  Boston              0


## Write 3 questions of your own and answer them

In [10]:
# Question 1: Which city has the highest number of customers

# Your code here

query_1 = '''
SELECT city, COUNT(customerNumber) AS num_customers
FROM customers
GROUP BY city
ORDER BY num_customers DESC
LIMIT 1;
'''

df = pd.read_sql_query(query_1, conn)
print(df)

  city  num_customers
0  NYC              5


In [11]:
# Question 2: Which product generates the highest revenue

# Your code here

query_2 = '''
SELECT p.productName, SUM(od.quantityOrdered * od.priceEach) AS total_revenue
FROM orderdetails od
JOIN products p ON od.productCode = p.productCode
GROUP BY p.productName
ORDER BY total_revenue DESC
LIMIT 1;
'''

df = pd.read_sql_query(query_2, conn)
print(df)

                   productName  total_revenue
0  1992 Ferrari 360 Spider red      276839.98


In [12]:
# Question 3: Which customers have not placed any orders

# Your code here

query_3 = '''
SELECT c.customerNumber, c.customerName
FROM customers c
LEFT JOIN orders o ON c.customerNumber = o.customerNumber
WHERE o.orderNumber IS NULL;
'''

df = pd.read_sql_query(query_3, conn)
print(df)

    customerNumber                    customerName
0              125              Havel & Zbyszek Co
1              168          American Souvenirs Inc
2              169               Porto Imports Co.
3              206      Asian Shopping Network, Co
4              223                 Natürlich Autos
5              237                   ANG Resellers
6              247        Messner Shopping Network
7              273               Franken Gifts, Co
8              293               BG&E Collectables
9              303                Schuyler Imports
10             307                Der Hund Imports
11             335       Cramer Spezialitäten, Ltd
12             348           Asian Treasures, Inc.
13             356            SAR Distributors, Co
14             361                 Kommission Auto
15             369          Lisboa Souveniers, Inc
16             376           Precious Collectables
17             409  Stuttgart Collectable Exchange
18             443        Feuer

## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [14]:
# Your code here

#Product name soled by each employee
query_4 = '''
SELECT e.firstName, e.lastName, p.productName
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
ORDER BY e.lastName, e.firstName, p.productName;
'''

df = pd.read_sql_query(query_4, conn)
print(df)

     firstName lastName                productName
0         Loui   Bondur      18th century schooner
1         Loui   Bondur     1900s Vintage Bi-Plane
2         Loui   Bondur    1900s Vintage Tri-Plane
3         Loui   Bondur          1903 Ford Model A
4         Loui   Bondur          1903 Ford Model A
...        ...      ...                        ...
2991    George   Vanauf                The Titanic
2992    George   Vanauf                The Titanic
2993    George   Vanauf  The USS Constitution Ship
2994    George   Vanauf  The USS Constitution Ship
2995    George   Vanauf  The USS Constitution Ship

[2996 rows x 3 columns]


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [16]:
# Your code here

#Number of products old by each employee
query_5 = '''
SELECT e.firstName, e.lastName, SUM(od.quantityOrdered) AS total_products_sold
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY e.firstName, e.lastName
ORDER BY e.lastName, e.firstName;
'''

df = pd.read_sql_query(query_5, conn)
print(df)

   firstName   lastName  total_products_sold
0       Loui     Bondur                 6186
1      Larry       Bott                 8205
2     Pamela   Castillo                 9290
3      Julie   Firrelli                 4227
4       Andy     Fixter                 6246
5     Martin     Gerard                 4180
6     Gerard  Hernandez                14231
7     Leslie   Jennings                11854
8      Barry      Jones                 7486
9      Peter      Marsh                 6632
10      Mami      Nishi                 4923
11     Steve  Patterson                 5561
12    Leslie   Thompson                 4056
13  Foon Yue      Tseng                 5016
14    George     Vanauf                 7423


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [18]:
# Your code here

# Employees who sold more than 200 different products
query_6 = '''
SELECT e.firstName, e.lastName, COUNT(DISTINCT p.productCode) AS unique_products_sold
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
GROUP BY e.firstName, e.lastName
HAVING unique_products_sold > 200
ORDER BY unique_products_sold DESC;
'''

df = pd.read_sql_query(query_6, conn)
print(df)

Empty DataFrame
Columns: [firstName, lastName, unique_products_sold]
Index: []


In [19]:
# Employees who sold highest number of different products

query_7 = '''
SELECT e.firstName, e.lastName, COUNT(DISTINCT p.productCode) AS unique_products_sold
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
GROUP BY e.firstName, e.lastName
ORDER BY unique_products_sold DESC;
'''

df = pd.read_sql_query(query_7, conn)
print(df)

   firstName   lastName  unique_products_sold
0     Gerard  Hernandez                   109
1     Leslie   Jennings                   107
2       Loui     Bondur                   101
3     Pamela   Castillo                   100
4      Barry      Jones                    98
5      Larry       Bott                    97
6      Peter      Marsh                    97
7     George     Vanauf                    96
8      Steve  Patterson                    95
9       Andy     Fixter                    82
10      Mami      Nishi                    81
11     Julie   Firrelli                    80
12    Martin     Gerard                    78
13  Foon Yue      Tseng                    74
14    Leslie   Thompson                    73


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!